In [5]:
!pip install pdfplumber pyhwp

  Using cached pdfplumber-0.11.7-py3-none-any.whl (60 kB)
  Using cached pyhwp-0.1b15.tar.gz (218 kB)
  Preparing metadata (setup.py) ... done
  Using cached pdfminer_six-20250506-py3-none-any.whl (5.6 MB)
  Using cached pypdfium2-4.30.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.8 MB)
Using legacy 'setup.py install' for pyhwp, since package 'wheel' is not installed.
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/opt/jhub-venv/lib/python3.10/site-packages/pypdfium2'
Check the permissions.



In [9]:
%pip install hwp5 pyhwp

ERROR: Could not find a version that satisfies the requirement hwp5 (from versions: none)
ERROR: No matching distribution found for hwp5
Note: you may need to restart the kernel to use updated packages.


In [4]:
import subprocess
result = subprocess.run(['/home/spai0323/myenv/bin/hwp5txt', '../data/raw/files/(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.hwp'], capture_output=True, text=True)
text = result.stdout
print(text[:500])






제 안 요 청 서





<표>





2024년 04월




<표>






<표>



<표>

□ 사 업 명 : 통합 정보시스템 구축 사전 컨설팅
□ 사업기간 : 계약체결일로부터 ~ 2024년 11월 29일
□ 사업예산 : 50,000,000(금 오천만원/VAT포함)
□ 사업추진방식 : 제한경쟁입찰(협상에 의한 계약)


<표>

 ㅇ 미술진흥법 제23조에 의거 통합미술정보시스템 구축‧운영
 ㅇ 통합미술정보시스템으로의 시스템 개편 및 고도화 전략 수립을 위한 기반 마련 필요성 대두 
 ㅇ 이용자 기반의 통합미술정보시스템 구축 방향 수립을 위한 사전 분석

<표>

 ㅇ 환경 및 현황분석
 ㅇ 통합미술정보시스템 구축 데이터 분석 및 설계
 ㅇ 통합정보시스템 벤치마킹 사례 조사
 ㅇ 비전 및 전략 수립 
 ㅇ 통합미술정보시스템 서비스 개발

<표>



<표>

□ 현재 시각예술정보시스템 한국미술시장정보시스템을 운영하고 있음

<표>

□ 한국 미술시장 정보 시스템


In [ ]:
import subprocess
import json
import os
import re

# hwp5txt 절대 경로
HWP5TXT_PATH = "/home/spai0323/myenv/bin/hwp5txt"

def extract_hwp_text(hwp_path):
    """hwp5txt로 HWP 텍스트 추출"""
    try:
        result = subprocess.run(
            [HWP5TXT_PATH, hwp_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True
        )
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"[실패] {hwp_path} : {e.stderr}")
        return None

def parse_text_and_tables(text):
    """
    본문과 표 구분 후, 표는 행/열 단위로 구조화
    """
    blocks = re.split(r'(<표>)', text)
    parsed = {"paragraphs": [], "tables": []}
    current_table = None

    for block in blocks:
        block = block.strip()
        if not block:
            continue
        if block == "<표>":
            # 새 테이블 시작
            current_table = []
            parsed["tables"].append(current_table)
        else:
            if current_table is not None:
                # 표의 내용은 hwp5txt에서는 '\n'과 '\t'로 구분
                rows = [row.split("\t") for row in block.split("\n") if row.strip()]
                current_table.extend(rows)
                current_table = None  # 표 끝
            else:
                # 일반 문단
                parsed["paragraphs"].append(block)
    return parsed

def hwp_folder_to_json(folder_path, output_folder):
    """폴더 내 모든 HWP를 JSON으로 변환"""
    os.makedirs(output_folder, exist_ok=True)
    hwp_files = [f for f in os.listdir(folder_path) if f.endswith(".hwp")]

    for hwp_file in hwp_files:
        hwp_path = os.path.join(folder_path, hwp_file)
        text = extract_hwp_text(hwp_path)
        if text is None:
            continue
        parsed = parse_text_and_tables(text)
        data = {
            "filename": hwp_file,
            "content": parsed
        }
        json_filename = os.path.splitext(hwp_file)[0] + ".json"
        json_path = os.path.join(output_folder, json_filename)
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"[완료] {hwp_file} -> {json_filename}")

# 사용 예시
if __name__ == "__main__":
    raw_folder = "../data/raw/files"
    processed_folder = "../data/processed"

    hwp_folder_to_json(raw_folder, processed_folder)
    print("모든 HWP 파일이 data/processed/json_output/에 JSON으로 변환되었습니다.")

[완료] 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.hwp -> 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.json
[완료] 케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 .hwp -> 케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 .json
[완료] 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.hwp -> 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.json
[완료] 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.hwp -> 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.json
[완료] 조선대학교_(재공고)2024 조선대학교 SW중심대학 사업관리시스템(WeHub) 구.hwp -> 조선대학교_(재공고)2024 조선대학교 SW중심대학 사업관리시스템(WeHub) 구.json
[완료] 국민연금공단_2024년 이러닝시스템 운영 용역.hwp -> 국민연금공단_2024년 이러닝시스템 운영 용역.json
[완료] 한국보건산업진흥원_의료기기산업 종합정보시스템(정보관리기관) 기능.hwp -> 한국보건산업진흥원_의료기기산업 종합정보시스템(정보관리기관) 기능.json
[완료] 울산광역시_2024년 버스정보시스템 확대 구축 및 기능개선 용역.hwp -> 울산광역시_2024년 버스정보시스템 확대 구축 및 기능개선 용역.json
[완료] BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).hwp -> BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).json
[완료] 부산관광공사_경영정보시스템 기능개선.hwp -> 부산관광공사_경영정보시스템 기능개선.json
[완료] 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.hwp -> 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.json
[

In [ ]:
import os
import json
import pdfplumber

def extract_pdf_text(pdf_path):
    """PDF 텍스트 추출 (페이지별로 리스트 구조화)"""
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"{pdf_path} 파일이 존재하지 않습니다.")

    pages_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""
            pages_text.append(text.strip())
    return pages_text

def pdf_folder_to_json(raw_folder, processed_folder):
    """폴더 내 모든 PDF를 JSON으로 변환"""
    json_output_folder = os.path.join(processed_folder, "json_output")
    os.makedirs(json_output_folder, exist_ok=True)

    pdf_files = [f for f in os.listdir(raw_folder) if f.endswith(".pdf")]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(raw_folder, pdf_file)
        pages_text = extract_pdf_text(pdf_path)
        data = {
            "filename": pdf_file,
            "pages": pages_text
        }
        json_filename = os.path.splitext(pdf_file)[0] + ".json"
        json_path = os.path.join(json_output_folder, json_filename)
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"[완료] {pdf_file} -> {json_filename}")

# 사용 예시
if __name__ == "__main__":
    raw_folder = "../data/raw/files"
    processed_folder = "../data/processed"

    pdf_folder_to_json(raw_folder, processed_folder)
    print("모든 PDF 파일이 data/processed/json_output/에 JSON으로 변환되었습니다.")

In [ ]:
import json
import os

def read_specific_json(folder_path, file_name):
    """
    지정된 폴더에서 특정 JSON 파일의 내용을 읽어 출력합니다.
    """
    file_path = os.path.join(folder_path, file_name)

    # 파일이 존재하는지 확인
    if not os.path.isfile(file_path):
        print(f"오류: '{file_path}' 파일을 찾을 수 없습니다.")
        return

    print(f"--- 파일: {file_name} ---")
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            # JSON 내용을 보기 좋게 출력
            print(json.dumps(data, indent=2, ensure_ascii=False))
    except json.JSONDecodeError as e:
        print(f"  [오류] JSON 디코딩 실패: {e}")
    except Exception as e:
        print(f"  [오류] 파일을 읽는 중 예외 발생: {e}")
    print("-" * (len(file_name) + 12)) # 구분선

# 사용 예시:
# 1. JSON 파일이 저장된 폴더 경로를 지정합니다.
json_folder_path = "../data/processed/json_output"

# 2. 읽고 싶은 특정 JSON 파일의 이름을 지정합니다.
#    예시: "광주과학기술원_학사시스템 기능개선 사업.json"
target_file = "(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.json"

read_specific_json(json_folder_path, target_file)

In [ ]:
import json
import os

def read_all_json(folder_path):
    """
    지정된 폴더 내 모든 JSON 파일의 내용을 읽어 출력합니다.
    """
    # 폴더가 존재하는지 확인
    if not os.path.isdir(folder_path):
        print(f"오류: '{folder_path}' 폴더를 찾을 수 없습니다.")
        return

    # 폴더 내 모든 파일 목록 가져오기
    json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

    if not json_files:
        print(f"'{folder_path}' 폴더에 JSON 파일이 없습니다.")
        return

    for file_name in json_files:
        file_path = os.path.join(folder_path, file_name)
        print(f"--- 파일: {file_name} ---")
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                # JSON 내용을 보기 좋게 출력
                print(json.dumps(data, indent=2, ensure_ascii=False))
        except json.JSONDecodeError as e:
            print(f"  [오류] JSON 디코딩 실패: {e}")
        except Exception as e:
            print(f"  [오류] 파일을 읽는 중 예외 발생: {e}")
        print("-" * (len(file_name) + 12)) # 구분선

# 사용 예시: 이전 코드에서 생성된 JSON 파일이 있는 폴더 경로
json_folder_path = "../data/processed/json_output/"
read_all_json(json_folder_path)

In [ ]:
import os
import json
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_json_text(json_folder):
    """폴더 내 모든 JSON 파일에서 텍스트 불러오기"""
    texts = []
    json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]

    for jf in json_files:
        path = os.path.join(json_folder, jf)
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # HWP 구조: paragraphs + tables
        content = data.get("content") or {}
        if content:
            # 문단 텍스트
            paragraphs = content.get("paragraphs", [])
            texts.extend(paragraphs)
            # 테이블 텍스트 (행·열 구조를 문자열로 변환)
            tables = content.get("tables", [])
            for table in tables:
                table_text = "\n".join(["\t".join(row) for row in table])
                texts.append(table_text)
        else:
            # PDF 구조: pages
            pages = data.get("pages", [])
            texts.extend(pages)

    return texts

def chunk_texts(texts, chunk_size=500, chunk_overlap=50):
    """텍스트 청킹"""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_text("\n".join(texts))
    return chunks

# 사용 예시
if __name__ == "__main__":
    json_folder = "projectmission2/data/processed/json_output"

    all_texts = load_json_text(json_folder)
    chunks = chunk_texts(all_texts, chunk_size=500, chunk_overlap=50)

    print(f"총 {len(chunks)}개의 청크 생성됨")
    print("첫 번째 청크 예시:\n", chunks[0])